In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import spotipy
import playlist
import config
import time
import create_dataframes
from sklearn.metrics import pairwise
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from spotipy.oauth2 import SpotifyOAuth

In [2]:
#create spotipy object to interact with spotify web API
from spotipy.oauth2 import SpotifyClientCredentials
sp = spotipy.Spotify(
    auth_manager=SpotifyClientCredentials
        (client_id=config.SPOTIFY_CLIENT_ID,
        client_secret=config.SPOTIFY_SECRET
        ))

avg_audio_values = pd.DataFrame(playlist1.normalized_numeric_features.loc[:,0:].mean(axis=0))
was changed from using just the normalized audio features to using all the normalized numeric features

In [3]:
#playlist_link, final_playlist_len = create_dataframes.get_user_playlist()
playlist_link, final_playlist_len = 'https://open.spotify.com/playlist/37i9dQZEVXcNDMTzEMNNE1', 10
playlist1 = playlist.Playlist(playlist_link, sp)
artist_counts, artist_id_counts = playlist1.get_artist_counts() 
avg_audio_values = pd.DataFrame(playlist1.normalized_numeric_features.loc[:,0:].mean(axis=0))
artists_search = create_dataframes.artist_search_results(artist_counts.columns)
recs = playlist1.get_recommendations(artists_search, avg_audio_values, k=5)
#create_dataframes.create_new_playlist(recs, final_playlist_len)

In [4]:
avg_audio_values

,0
0,0.601267
1,0.463976
2,0.457576
3,0.673927
4,0.566667
5,0.450619
6,0.257194
7,0.070887
8,0.274420
9,0.473408


- standardize ratings + audio features  
- compute PCA on the dataframe with the standardized numeric features  
- check how many components explain 80% ish of the variance, & keep just those (set playlist1.normalized_audio_features = PCA reduced data)  
- compute the average value of the selected components  
- perform artists search  
- NORMALIZE THE AUDIO FEATURES FROM THE SONGS IN THE SEARCH RESULT**** (add a PCA flag to playlist.get_recommendations which will perform PCA transformation on the data it gets back after line 192 in playlist.py)  
- find KNN with PCA performed data

In [5]:
print(playlist1.audio_features.shape)
playlist1.audio_features

(30, 13)


,track_id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
0,2XHzzp1j4IfTNp1FTn7YFg,0.669,0.634,11,-6.476,1,0.0327,0.01250,0.000000,0.0946,0.496,124.906,255053
1,01A7PEPSnmtixFPfB2UTal,0.793,0.631,11,-6.109,0,0.0998,0.04060,0.000478,0.2070,0.231,119.971,226693
2,6AaOtHsKd195ec0Y4kC9ER,0.721,0.607,0,-7.078,0,0.2770,0.09580,0.000000,0.2900,0.590,76.987,206267
3,63M8PK8yavNITSViKUB62p,0.752,0.766,1,-5.110,1,0.2510,0.33800,0.000000,0.0861,0.501,141.820,330770
4,17Lmf8pUgrRrEKIKkuZirE,0.799,0.499,11,-7.785,1,0.1820,0.01030,0.000000,0.1840,0.130,95.059,212246
5,2LvRR121MWFmmEGkuV2vQP,0.919,0.506,1,-7.140,1,0.3040,0.13200,0.000000,0.0771,0.273,97.465,220240
6,1s9hBKw37JvYV9KYiR9oef,0.744,0.539,10,-6.757,1,0.0596,0.14700,0.000000,0.2630,0.509,145.915,173213
7,1UYLxZIL4IicS6TaPUZFTH,0.893,0.635,2,-9.000,1,0.0514,0.02010,0.000003,0.1460,0.304,127.980,285234
8,5BviN9XWqCYE88AOJg6nEK,0.801,0.521,4,-9.871,0,0.2800,0.01360,0.000000,0.1790,0.412,108.025,334360
9,4fUh9VvKW4shhWN5LfN1Ba,0.762,0.611,9,-5.462,0,0.0876,0.12200,0.000315,0.1410,0.386,99.934,206760


In [6]:
print(playlist1.normalized_audio_features.shape) #make sure its the same as the nontransformed audio data
playlist1.normalized_audio_features

(30, 13)


,track_id,0,1,2,3,4,5,6,7,8,9,10,11
0,2XHzzp1j4IfTNp1FTn7YFg,0.436,0.528440,1.000000,0.723185,1.0,0.000000,0.018354,0.000000,0.072468,0.592357,0.489594,0.489199
1,01A7PEPSnmtixFPfB2UTal,0.684,0.522936,1.000000,0.761035,0.0,0.179268,0.084902,1.000000,0.254287,0.170382,0.439172,0.319677
2,6AaOtHsKd195ec0Y4kC9ER,0.540,0.478899,0.000000,0.661097,0.0,0.652685,0.215631,0.000000,0.388547,0.742038,0.000000,0.197580
3,63M8PK8yavNITSViKUB62p,0.602,0.770642,0.090909,0.864068,1.0,0.583222,0.789224,0.000000,0.058719,0.600318,0.662406,0.941797
4,17Lmf8pUgrRrEKIKkuZirE,0.696,0.280734,1.000000,0.588181,1.0,0.398878,0.013144,0.000000,0.217082,0.009554,0.184644,0.233320
5,2LvRR121MWFmmEGkuV2vQP,0.936,0.293578,0.090909,0.654703,1.0,0.724820,0.301362,0.000000,0.044160,0.237261,0.209226,0.281104
6,1s9hBKw37JvYV9KYiR9oef,0.586,0.354128,0.909091,0.694204,1.0,0.071867,0.336886,0.000000,0.344872,0.613057,0.704245,0.000000
7,1UYLxZIL4IicS6TaPUZFTH,0.884,0.530275,0.181818,0.462871,1.0,0.049960,0.036353,0.006297,0.155613,0.286624,0.521001,0.669606
8,5BviN9XWqCYE88AOJg6nEK,0.700,0.321101,0.363636,0.373040,0.0,0.660700,0.020959,0.000000,0.208994,0.458599,0.317119,0.963256
9,4fUh9VvKW4shhWN5LfN1Ba,0.622,0.486239,0.818182,0.827764,0.0,0.146674,0.277679,0.658996,0.147525,0.417197,0.234452,0.200527


In [7]:
print(playlist1.numeric_features.shape) 
playlist1.numeric_features
#includes popularity

(30, 14)


,track_id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,popularity
0,2XHzzp1j4IfTNp1FTn7YFg,0.669,0.634,11,-6.476,1,0.0327,0.01250,0.000000,0.0946,0.496,124.906,255053,79
1,01A7PEPSnmtixFPfB2UTal,0.793,0.631,11,-6.109,0,0.0998,0.04060,0.000478,0.2070,0.231,119.971,226693,57
2,6AaOtHsKd195ec0Y4kC9ER,0.721,0.607,0,-7.078,0,0.2770,0.09580,0.000000,0.2900,0.590,76.987,206267,50
3,63M8PK8yavNITSViKUB62p,0.752,0.766,1,-5.110,1,0.2510,0.33800,0.000000,0.0861,0.501,141.820,330770,52
4,17Lmf8pUgrRrEKIKkuZirE,0.799,0.499,11,-7.785,1,0.1820,0.01030,0.000000,0.1840,0.130,95.059,212246,56
5,2LvRR121MWFmmEGkuV2vQP,0.919,0.506,1,-7.140,1,0.3040,0.13200,0.000000,0.0771,0.273,97.465,220240,67
6,1s9hBKw37JvYV9KYiR9oef,0.744,0.539,10,-6.757,1,0.0596,0.14700,0.000000,0.2630,0.509,145.915,173213,38
7,1UYLxZIL4IicS6TaPUZFTH,0.893,0.635,2,-9.000,1,0.0514,0.02010,0.000003,0.1460,0.304,127.980,285234,54
8,5BviN9XWqCYE88AOJg6nEK,0.801,0.521,4,-9.871,0,0.2800,0.01360,0.000000,0.1790,0.412,108.025,334360,45
9,4fUh9VvKW4shhWN5LfN1Ba,0.762,0.611,9,-5.462,0,0.0876,0.12200,0.000315,0.1410,0.386,99.934,206760,54


In [8]:
print(playlist1.normalized_numeric_features.shape) 
playlist1.normalized_numeric_features

(30, 14)


,track_id,0,1,2,3,4,5,6,7,8,9,10,11,12
0,2XHzzp1j4IfTNp1FTn7YFg,0.436,0.528440,1.000000,0.723185,1.0,0.000000,0.018354,0.000000,0.072468,0.592357,0.489594,0.489199,1.000000
1,01A7PEPSnmtixFPfB2UTal,0.684,0.522936,1.000000,0.761035,0.0,0.179268,0.084902,1.000000,0.254287,0.170382,0.439172,0.319677,0.531915
2,6AaOtHsKd195ec0Y4kC9ER,0.540,0.478899,0.000000,0.661097,0.0,0.652685,0.215631,0.000000,0.388547,0.742038,0.000000,0.197580,0.382979
3,63M8PK8yavNITSViKUB62p,0.602,0.770642,0.090909,0.864068,1.0,0.583222,0.789224,0.000000,0.058719,0.600318,0.662406,0.941797,0.425532
4,17Lmf8pUgrRrEKIKkuZirE,0.696,0.280734,1.000000,0.588181,1.0,0.398878,0.013144,0.000000,0.217082,0.009554,0.184644,0.233320,0.510638
5,2LvRR121MWFmmEGkuV2vQP,0.936,0.293578,0.090909,0.654703,1.0,0.724820,0.301362,0.000000,0.044160,0.237261,0.209226,0.281104,0.744681
6,1s9hBKw37JvYV9KYiR9oef,0.586,0.354128,0.909091,0.694204,1.0,0.071867,0.336886,0.000000,0.344872,0.613057,0.704245,0.000000,0.127660
7,1UYLxZIL4IicS6TaPUZFTH,0.884,0.530275,0.181818,0.462871,1.0,0.049960,0.036353,0.006297,0.155613,0.286624,0.521001,0.669606,0.468085
8,5BviN9XWqCYE88AOJg6nEK,0.700,0.321101,0.363636,0.373040,0.0,0.660700,0.020959,0.000000,0.208994,0.458599,0.317119,0.963256,0.276596
9,4fUh9VvKW4shhWN5LfN1Ba,0.622,0.486239,0.818182,0.827764,0.0,0.146674,0.277679,0.658996,0.147525,0.417197,0.234452,0.200527,0.468085


In [9]:
playlist1.df_playlist

,index,song_title,artist,artist_id,album,track_id,popularity,genres,track_id,danceability,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,popularity
0,1,Love Me,"[Lil Wayne, Drake, Future]","[55Aa2cqylxrFIXC767Z865, 3TVXtAsR1Inumwj472S9r...",I Am Not A Human Being II (Deluxe),2XHzzp1j4IfTNp1FTn7YFg,79,"[toronto rap, pop rap, rap, atl hip hop, trap,...",2XHzzp1j4IfTNp1FTn7YFg,0.669,...,-6.476,1,0.0327,0.01250,0.000000,0.0946,0.496,124.906,255053,79
1,2,Buy The World,"[Mike WiLL Made-It, Lil Wayne, Kendrick Lamar,...","[0NWbwDZY1VkRqFafuQm6wk, 55Aa2cqylxrFIXC767Z86...",Buy The World,01A7PEPSnmtixFPfB2UTal,57,"[atl hip hop, rap, pop rap, conscious hip hop,...",01A7PEPSnmtixFPfB2UTal,0.793,...,-6.109,0,0.0998,0.04060,0.000478,0.2070,0.231,119.971,226693,57
2,3,DnF (feat. Drake & Future),"[Preme, Drake, Future]","[0bdJZl7TDeiymDYzMJnVh2, 3TVXtAsR1Inumwj472S9r...",DnF (feat. Drake & Future),6AaOtHsKd195ec0Y4kC9ER,50,"[toronto rap, pop rap, canadian trap, rap, atl...",6AaOtHsKd195ec0Y4kC9ER,0.721,...,-7.078,0,0.2770,0.09580,0.000000,0.2900,0.590,76.987,206267,50
3,4,Schemin Up (feat. Drake and P. Reign),"[OB OBrien, Drake, Preme]","[4thGcsymOK5oc43gJVtAqZ, 3TVXtAsR1Inumwj472S9r...",Schemin Up (feat. Drake and P. Reign),63M8PK8yavNITSViKUB62p,52,"[toronto rap, pop rap, rap, canadian trap, tra...",63M8PK8yavNITSViKUB62p,0.752,...,-5.110,1,0.2510,0.33800,0.000000,0.0861,0.501,141.820,330770,52
4,5,I Just Wanna Party,"[YG, ScHoolboy Q, Jay Rock]","[0A0FS04o6zMoto8OKPsDwY, 5IcR3N7QB1j6KBL8eImZ8...",My Krazy Life,17Lmf8pUgrRrEKIKkuZirE,56,"[pop rap, rap, conscious hip hop, underground ...",17Lmf8pUgrRrEKIKkuZirE,0.799,...,-7.785,1,0.1820,0.01030,0.000000,0.1840,0.130,95.059,212246,56
5,6,HeadBand (feat. 2 Chainz),"[B.o.B, 2 Chainz]","[5ndkK3dpZLKtBklKjxNQwT, 17lzZA2AlOHwCwFALHttmp]",Underground Luxury,2LvRR121MWFmmEGkuV2vQP,67,"[atl hip hop, pop rap, rap, trap, hip hop, sou...",2LvRR121MWFmmEGkuV2vQP,0.919,...,-7.140,1,0.3040,0.13200,0.000000,0.0771,0.273,97.465,220240,67
6,7,Fefe On The Block,[Stunt Taylor],[6OlcRxdAiJfOY2SiexBMej],Stunt'n On Turbo,1s9hBKw37JvYV9KYiR9oef,38,"[chicago bop, chicago rap]",1s9hBKw37JvYV9KYiR9oef,0.744,...,-6.757,1,0.0596,0.14700,0.000000,0.2630,0.509,145.915,173213,38
7,8,Blasé (feat. Future & Rae Sremmurd),"[Ty Dolla $ign, Future, Rae Sremmurd]","[7c0XG5cIJTrrAgEC3ULPiq, 1RyvyyTE3xzB2ZywiAwp0...",Free TC,1UYLxZIL4IicS6TaPUZFTH,54,"[pop rap, atl hip hop, rap, trap, hip hop, r&b...",1UYLxZIL4IicS6TaPUZFTH,0.893,...,-9.000,1,0.0514,0.02010,0.000003,0.1460,0.304,127.980,285234,54
8,9,100,"[The Game, Drake]","[0NbfKEOTQCcwd6o7wSDOHI, 3TVXtAsR1Inumwj472S9r4]",The Documentary 2,5BviN9XWqCYE88AOJg6nEK,45,"[toronto rap, pop rap, rap, hip hop, southern ...",5BviN9XWqCYE88AOJg6nEK,0.801,...,-9.871,0,0.2800,0.01360,0.000000,0.1790,0.412,108.025,334360,45
9,10,Be Real (feat. DeJ Loaf),"[Kid Ink, DeJ Loaf]","[6KZDXtSj0SzGOV705nNeh3, 7kFfY4UjNdNyaeUgLIEbIF]",Full Speed,4fUh9VvKW4shhWN5LfN1Ba,54,"[pop r&b, pop rap, rap, trap queen, trap, r&b,...",4fUh9VvKW4shhWN5LfN1Ba,0.762,...,-5.462,0,0.0876,0.12200,0.000315,0.1410,0.386,99.934,206760,54


In [10]:
pca_obj, playlist1.pca_transformed_features = playlist1.pca_transform(playlist1.normalized_numeric_features.loc[:,0:])

In [11]:
pca_obj.explained_variance_

array([0.30594016, 0.15391683, 0.13724652, 0.10479275, 0.09413014,
       0.07207239, 0.06556575, 0.05663633, 0.03776728, 0.03538319,
       0.01973168, 0.01348896, 0.00786931])

In [12]:
len(pca_obj.explained_variance_)

13

In [13]:
sum(pca_obj.explained_variance_[:6]) #reduce playlist1.pca_transformed_features to 5 or 6 components

0.8680987951170095

In [14]:
#automate the process to find # of components that explain 75% of the variance in the data
# should be 'automated' because this may vary per playlist
def choose_components(pca, pct):
    '''Will find the number of components in PCA necessary to explain pct of the variance in the dataset
       -------------------------------------------
       parameters:
       - pca: pandas.decomposition.PCA object
       - pct: % variance youd like explained by the components
       -------------------------------------------
       returns:
       - num_components: number of components to explain desired % variance
    '''
    
    for num_components in range(0, len(pca.explained_variance_)):
        var_explained = sum(pca.explained_variance_[:num_components])
        if var_explained >= pct:
            return num_components
    return -1

In [15]:
components = choose_components(pca_obj, .85)
components

6

In [16]:
#reduce the pca data
playlist1.pca_transformed_features = pd.DataFrame(playlist1.pca_transformed_features.iloc[:, :components])
playlist1.pca_transformed_features

,0,1,2,3,4,5
0,-0.310691,0.783396,0.100749,0.048871,0.297630,0.364042
1,0.638580,0.712854,-0.500507,0.056164,-0.392342,0.385841
2,0.501253,-0.499147,-0.116262,-0.110061,-0.097786,-0.004071
3,-0.486820,-0.505744,0.452408,0.193701,0.285574,0.503326
4,-0.420821,0.666693,-0.183224,-0.345210,-0.130369,-0.154391
5,-0.609557,-0.158517,-0.236944,-0.356428,0.144582,0.069139
6,-0.334146,0.450530,0.191466,0.437847,-0.251508,-0.402911
7,-0.540953,-0.115519,-0.295052,-0.057181,0.116795,0.172252
8,0.409572,-0.428562,-0.157281,-0.559135,-0.190773,0.279345
9,0.544823,0.441660,-0.384768,0.223262,-0.397805,0.348767


In [17]:
recs

,artist,track,track_id,similarity
0,Lil Wayne,Hittas,5s6pKnF3ZPtuqvEx1i1V7U,0.939879
1,Lil Wayne,Mona Lisa (feat. Kendrick Lamar),0dbTQYW3Ad1FTzIA9t90E8,0.929644
2,Lil Wayne,No Worries,6bdFbw9THAMceZtkPWXu8e,0.921159
3,Lil Wayne,Drop The World,3e21cX0CVwzkQXiHz7WUQZ,0.916111
4,Lil Wayne,Mrs. Officer,0EHR9OObFtjlhQB8wSt1m7,0.911495
...,...,...,...,...
270,JAY-Z,Empire State Of Mind,2igwFfvr1OAGX9SKDCPBwO,0.937264
271,JAY-Z,Neck & Wrist (feat. JAY-Z & Pharrell Williams),7kQJCw0ZkvHgfJqRwPblmG,0.922061
272,JAY-Z,Ni**as In Paris,4Li2WHPkuyCdtmokzW2007,0.919346
273,JAY-Z,Heartbreaker (feat. Jay-Z),0jsANwwkkHyyeNyuTFq2XO,0.918459


In [18]:
playlist1.pca_transformed_features.iloc[:,5]

0     0.364042
1     0.385841
2    -0.004071
3     0.503326
4    -0.154391
5     0.069139
6    -0.402911
7     0.172252
8     0.279345
9     0.348767
10   -0.005960
11    0.066486
12    0.327193
13   -0.200116
14    0.203645
15    0.073123
16    0.049104
17   -0.358666
18   -0.279588
19   -0.299324
20   -0.338365
21   -0.379722
22   -0.499380
23   -0.141745
24   -0.140255
25    0.185311
26   -0.142255
27    0.114949
28    0.128456
29    0.075772
Name: 5, dtype: float64

In [19]:
#recalculate average audio values on the pca reduced data
avg_audio_values = pd.DataFrame(playlist1.pca_transformed_features.mean(axis=0))

In [20]:
avg_audio_values

,0
0,2.220446e-17
1,2.405483e-17
2,1.850372e-18
3,4.348374e-17
4,8.511710e-17
5,-2.081668e-17


In [21]:
artists_search = create_dataframes.artist_search_results(artist_counts.columns)
recs = playlist1.get_recommendations(artists_search, avg_audio_values, pca_components=components, k=5)

In [22]:
recs

,artist,track,track_id,similarity
0,Lil Wayne,SMUCKERS (feat. Lil Wayne & Kanye West),078C2jXg7XsMgW78Gfx1JA,0.0
1,Lil Wayne,Down,6cmm1LMvZdB5zsCwX5BjqE,0.0
2,Lil Wayne,Beware,6bxUnsSGZCmoHHU5auwtps,0.0
3,Lil Wayne,6 Foot 7 Foot,5Qy6a5KzM4XlRxsNcGYhgH,0.0
4,Lil Wayne,Lollipop,4P7VFiaZb3xrXoqGwZXC3J,0.0
...,...,...,...,...
270,JAY-Z,4:44,1gT5TGwbkkkUliNzHRIGi1,0.0
271,JAY-Z,JAY-Z,684MKCXtMZF58P24BkGFkJ,0.0
272,JAY-Z,"GOD DID (feat. Rick Ross, Lil Wayne, Jay-Z, Jo...",2sOj9vyd6yiss9W1IK6chU,0.0
273,JAY-Z,Neck & Wrist (feat. JAY-Z & Pharrell Williams),7kQJCw0ZkvHgfJqRwPblmG,0.0


why are similarities = 0...

In [23]:
l = [-4.870593456842883e-17, -2.1693136948257682e-17, 1.92738037764736e-17, -5.4953127117554664e-17, -3.851701114541537e-17, -4.703354558138195e-17, -4.232986903522912e-17, -3.376205137093322e-17, 5.729488745054256e-18, -3.8245643962455447e-17, -6.302553863857029e-18, 2.456685981015234e-17, -4.427187178860939e-17, -2.3034266875922977e-17, -1.7857014815993827e-18, -2.3401755504785702e-17, 3.6637632269869525e-17, -3.344708781483712e-17, 6.362684330101618e-17, -3.442172624212567e-17, -4.507372592724885e-17, -3.976255404469413e-17, 6.55398984447831e-17, -5.276580527433639e-17, 6.603219881947715e-17, -2.192936836359534e-17, 6.69623141049022e-17, -5.3623560922766583e-17, 6.69623141049022e-17, -2.85407691716247e-17, 7.60854490150577e-17, -7.453324515636927e-17, 3.439532472056274e-17, -2.5963331616325057e-18, 4.567266939320712e-17, 5.606903253104433e-17, 2.974710419131052e-17, 8.592770652473752e-19, 2.640192323269679e-18, 4.986379424484309e-18, -5.543518146210285e-17, -1.0731464603142767e-17, 1.8149203357915017e-17, 4.050513002089836e-17, -3.519374397313274e-17, -6.270715846914461e-17, -4.9048669010730297e-17, 4.9407019086493826e-17, 4.3136237181538396e-17, -1.7168268715241834e-17]
max(l)

7.60854490150577e-17

because values in similarities are tiny. how? if we used cosine distance

try using standard scalers  
- changed line 206 in create_dataframes.py to use standardscaler instead of maxscaler.
- should try using another distance measure too